<a href="https://colab.research.google.com/github/c-chaitanya/penny/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup as bs

In [2]:
try:
    import shutil
    current_file_path = os.getcwd()
    path = os.path.join(current_file_path, "Astro")
    shutil.rmtree(path)
except:
    pass

In [6]:
df = pd.read_excel("Dataset - Penny test.xlsx", sheet_name="Astro")
try:
    # Downloading images to a folder and updating location in csv
    current_file_path = os.getcwd()
    folder_path = os.path.join(current_file_path, "Astro")
    os.mkdir(folder_path)

    df['Offline_image_path'] = ""

    for i in df['Image']:
        filename = i.split('/')[-1]
        path = os.path.join(folder_path, filename)
        urllib.request.urlretrieve(i, path)
        df.loc[df['Image']==i,'Offline_image_path'] = str(path)
except Exception as e:
    print(e)
    pass

[Errno 17] File exists: '/content/Astro'


In [7]:
# Scraping technical details about product
page = 0
for url in df['Identifier']:
    print(url)
    page +=1
    r = requests.get(url)
    if r.status_code != 200:
        print("page does not exist")
        df.loc[df['Identifier'] == url,'description'] = ""
        df.loc[df['Identifier'] == url,'specification'] = ""
        df.loc[df['Identifier'] == url,'part_breakdown'] = ""
        df.loc[df['Identifier'] == url,'product_info_sheet'] = ""
        df.loc[df['Identifier'] == url,'marketing_flyer'] = ""
        df.loc[df['Identifier'] == url,'related_products'] = ""

    else:
        print("========================================")
        content = r.content
        soup = bs(content, 'html.parser')

        try:
            item = soup.find_all('div', {'class': 'value'})[0].text
            df.loc[df['Identifier'] == url,'item'] = item
        except Exception as e:
            print("exception in item: ", e)
            df.loc[df['Identifier'] == url,'item'] = ""
        
        try:
            description = soup.find_all('div',  id="product.info.description")[0].text
            df.loc[df['Identifier'] == url,'description'] = description
        except Exception as e:
            print("exception in description: ", e)
            df.loc[df['Identifier'] == url,'description'] = ""

        try:
            specification = soup.find_all('div',  id="product.info.description.attribute")[0].text
            df.loc[df['Identifier'] == url,'specification'] = specification
        except Exception as e:
            print("exception in specification: ", e)
            df.loc[df['Identifier'] == url,'specification'] = specification

        try:
            part_breakdown = soup.find_all('div',  id="product.info.breakdown")
            link = part_breakdown[0].find_all('a', href=True)
            pdf_url = link[0].get('href')
            filename = "part_breakdown_" + pdf_url.split('/')[-1]
            path = os.path.join(folder_path, filename)
            urllib.request.urlretrieve(pdf_url, path)
            df.loc[df['Identifier'] == url,'part_breakdown'] = str(path)
        except Exception as e:
            print("exception in part_breakdown: ", e)
            df.loc[df['Identifier'] == url,'part_breakdown'] = ""

        try:
            product_info_sheet = soup.find_all('div',  id="product.info.sheet")
            link = product_info_sheet[0].find_all('a', href=True)
            pdf_url = link[0].get('href')
            filename = "product_info_sheet_" + pdf_url.split('/')[-1]
            path = os.path.join(folder_path, filename)
            urllib.request.urlretrieve(pdf_url, path)
            df.loc[df['Identifier'] == url,'product_info_sheet'] = str(path)
        except Exception as e:
            print("exception in product_info_sheet: ", e)
            df.loc[df['Identifier'] == url,'product_info_sheet'] = ""

        try:
            marketing_flyer = soup.find_all('div',  id="product.info.flyer")
            link = marketing_flyer[0].find_all('a', href=True)
            pdf_url = link[0].get('href')
            filename = "marketing_flyer_" + pdf_url.split('/')[-1]
            path = os.path.join(folder_path, filename)
            urllib.request.urlretrieve(pdf_url, path)
            df.loc[df['Identifier'] == url,'marketing_flyer'] = str(path)
        except Exception as e:
            print("exception in marketing_flyer: ", e)
            df.loc[df['Identifier'] == url,'marketing_flyer'] = ""
   
        try:
            related_txt = ""
            related_products = soup.find_all('li',  'item product product-item')
            for cell in related_products:
                cell1 = cell.find_all('a', href=True)
                for link in cell1:
                    l = link['href']
                    images = link.find_all('img', alt=True)
                    for image in images:
                        i = image['src']
                        related_text = i.split('/')[-1]
                        related_txt = related_txt + " item " + related_text.split('.')[0] + "\n link:" + l + "\n image:" + i + "\n\n"
            df.loc[df['Identifier'] == url,'related_products'] = str(related_txt)

        except Exception as e:
            print("exception in related_products: ", e)
            df.loc[df['Identifier'] == url,'related_products'] = ""

        try:
            #item = soup.find_all('a', {'class'='amazon action primary tocart'})[0].text
            buy_online = soup.find('a', {'class':'amazon action primary tocart'})['href']
            df.loc[df['Identifier'] == url,'buy_online'] = buy_online
        except Exception as e:
            print("exception in item: ", e)
            df.loc[df['Identifier'] == url,'buy_online'] = ""    

print("done")

http://www.astrotools.com/air-tools/drills-screwdriviers/3-8-reversible-air-drill-1-800rpm.html
page does not exist
http://www.astrotools.com/1-2-extra-heavy-duty-reversible-air-drill-500rpm.html
http://www.astrotools.com/industrial-1-4-air-die-grinder.html
exception in marketing_flyer:  list index out of range
http://www.astrotools.com/1-2-air-ratchet-wrench-50ft-lb-torque.html
exception in part_breakdown:  HTTP Error 404: Not Found
exception in product_info_sheet:  HTTP Error 404: Not Found
exception in marketing_flyer:  list index out of range
http://www.astrotools.com/1-2-super-duty-impact-wrench-twin-hammer.html
exception in part_breakdown:  HTTP Error 404: Not Found
exception in marketing_flyer:  list index out of range
http://www.astrotools.com/1-heavy-duty-air-impact-wrench-with-2-anvil.html
exception in marketing_flyer:  list index out of range
http://www.astrotools.com/1-heavy-duty-air-impact-wrench-with-6-anvil.html
exception in part_breakdown:  HTTP Error 404: Not Found
exc

In [8]:
df1 = df

# Scrapping Knipex part using selenium

In [9]:
df = pd.read_excel("Dataset - Penny test.xlsx", sheet_name="Knipex")

In [10]:
# Downloading images to a folder and updating location in csv
try:
    current_file_path = os.getcwd()
    folder_path = os.path.join(current_file_path, "Knipex")
    os.mkdir(folder_path)

    df['Offline_image_path'] = ""


    for i in df['Image']:
        filename = i.split('/')[-1]
        path = os.path.join(folder_path, filename)
        urllib.request.urlretrieve(i, path)
        df.loc[df['Image']==i,'Offline_image_path'] = str(path)
except Exception as e:
    print(e)

In [11]:
# Scraping technical details about product
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime

driver = webdriver.Firefox()
# Opening webpage
driver.get("https://www.knipex.com/nc/en/home/")

for _, row in df.iterrows():
    keyword = row['Identifier']
    if isinstance(keyword, datetime.datetime):
        df.loc[df['Identifier']==keyword, 'product_info'] = ""
        df.loc[df['Identifier']==keyword, 'Description'] = ""
        df.loc[df['Identifier']==keyword, 'Special'] = ""
        df.loc[df['Identifier']==keyword, 'applications'] = ""
        df.loc[df['Identifier']==keyword, 'pdf'] = ""
        df.loc[df['Identifier']==keyword, 'video'] = ""
        continue

    sub_category = row['Sub-Category']
    print("================================================")
    print(keyword)
    print(sub_category)
    # Finding the search bar
    print("Searchbar")
    searchbar = driver.find_element_by_id("s")
    # Cleaning already existing text in search bar
    searchbar.clear()
    # Entering keyword to search in searchbar
    print("entering data")
    searchbar.send_keys(keyword)
    searchbar.send_keys(Keys.RETURN)
    # Explicit wait for searched element to be available
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "searchout"))
        )
        # Searching for the matching category
        try:
            link = driver.find_element_by_link_text(sub_category)
        except:
            link = driver.find_element_by_xpath('//*[@id="searchout"]')
        # Clicking on returned link
        print("clicking")
        link.click()

        try:
            product = driver.find_element_by_class_name('pgfinish')
            #print(product.text)
            df.loc[df['Identifier']==keyword, 'product_info'] = product.text
        except Exception as e:
            print("product error: ", e)
            df.loc[df['Identifier']==keyword, 'product_info'] = ""
        print("========================")

        try:
            Description = driver.find_element_by_id('fragment-1')
            #print(Description.text)
            df.loc[df['Identifier']==keyword, 'Description'] = Description.text
        except Exception as e:
            print("description error: ", e)
            df.loc[df['Identifier']==keyword, 'Description'] = ""
        print("========================")

        try:
            html = driver.page_source
            soup = bs(html)
            Special = soup.find_all('div',  id="fragment-2")[0].text
            df.loc[df['Identifier']==keyword, 'Special'] = Special
        except Exception as e:
            print("special error: ", e)
            df.loc[df['Identifier']==keyword, 'Special'] = ""
        print("========================")

        try:
            html = driver.page_source
            soup = bs(html)
            applications = soup.find_all('div',  id="fragment-3")[0].text
            applications = soup.find_all('div',  {'class':"artdetail_anw_img"})
            app = ""
            for i in applications:
                app = app + " " + i.text + "\n"
            df.loc[df['Identifier']==keyword, 'applications'] = app
        except Exception as e:
            print("application error: ", e)
            df.loc[df['Identifier']==keyword, 'applications'] = ""
        print("========================")

        
        try:
            link = driver.find_element_by_link_text(keyword)
            link.click()
            
            html = driver.page_source
            soup = bs(html)
            technical_details = soup.find_all('div',  id="fragment-1")[0].text
            df.loc[df['Identifier']==keyword, 'technical_details'] = technical_details
        except Exception as e:
            print("technical_details error: ", e)
            df.loc[df['Identifier']==keyword, 'technical_details'] = ""
        print("========================")

        
        try:
            html = driver.page_source
            soup = bs(html)
            spare = soup.find_all('div', {'class':'pgJoins'})[0].text
            df.loc[df['Identifier']==keyword, 'spare-accessories'] = spare
        except Exception as e:
            print("technical_details error: ", e)
            df.loc[df['Identifier']==keyword, 'spare-accessories'] = ""
        print("========================")
        
        try:
            html = driver.page_source
            soup = bs(html)
            pdf = soup.find_all('div',  id="fragment-7")
            link = pdf[0].find_all('a', href=True)
            path = ""
            for i in link:
                pdf_url = i.get('href')
                filename = pdf_url.split('/')[-1]
                pat = os.path.join(folder_path, filename)
                urllib.request.urlretrieve("http://www.knipex.com"+pdf_url, pat)
                path = path + pat + "\n"
            df.loc[df['Identifier']==keyword, 'pdf'] = str(path)
        except Exception as e:
            print("pdf error: ", e)
            df.loc[df['Identifier']==keyword, 'pdf'] = ""
        print("========================")
        
        try:
            html = driver.page_source
            soup = bs(html)
            #video = soup.find_all('div',  id="fragment-6")
            video = soup.find_all('iframe')
            link = soup.find_all('iframe')[0]['src']
            df.loc[df['Identifier']==keyword, 'video'] = str(link)
        except Exception as e:
            print("video error: ", e)
            df.loc[df['Identifier']==keyword, 'video'] = ""
            driver.back()
        print("========================")

        # going back by 2 pages to the start page
        driver.back()
        driver.back()

    except Exception as e:
        print(e)
        continue

driver.quit()

ModuleNotFoundError: ignored

In [ ]:
df2 = df

In [ ]:
with pd.ExcelWriter('output.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Astro')
    df2.to_excel(writer, sheet_name='Knipex')